In [1]:
import mysql.connector
import pandas as pd

In [2]:
cnx = mysql.connector.connect(user='admin', password='gatechoms2424!',
                              host='database-1-instance-1.cebbxjvkth1w.us-east-1.rds.amazonaws.com',
                              database='opioids', use_pure=True)

In [3]:
### 1) Combined Fips codes for counties with years 2006-2012 to get a base frame
df_fips = pd.read_sql('SELECT * FROM lkup_fips_codes', cnx)

df_year = pd.DataFrame({'year':['2006', '2007', '2008', '2009', '2010', '2011', '2012']})

df_main = df_year.assign(key=1).merge(df_fips.assign(key=1), on='key').reset_index()[['year', 'fips_combined', 'state_abbr', 'county_name']]

df_main = df_main.rename({'fips_combined': 'fips'}, axis=1)
df_main = df_main.set_index(['year', 'fips'])

In [4]:
### 2) Pull in population by county by year
df_population = pd.read_sql('SELECT * FROM population', cnx)

df_population = df_population.rename({'yr': 'year', 'fips_combined_fk': 'fips'}, axis=1)
df_population['year'] = df_population['year'].astype(str)
df_population = df_population.set_index(['year', 'fips'])

#join with the main frame
df_main = df_main.join(df_population)

In [5]:
### 3) Pull in sales by county by year
df_sales = pd.read_sql('SELECT * FROM sales', cnx)

df_sales = df_sales.rename({'yr': 'year', 'fips_combined_fk': 'fips'}, axis=1)
df_sales['year'] = df_sales['year'].astype(str)
df_sales = df_sales.set_index(['year', 'fips'])

#join
df_main = df_main.join(df_sales)

#create per capita sales
df_main['sales_per_capita'] = df_main['sales'] / df_main['population']

In [6]:
### 4) Pull in median income by county by year
df_income = pd.read_sql('SELECT * FROM median_income', cnx)

df_income = df_income.rename({'yr': 'year', 'fips_combined_fk': 'fips'}, axis=1)
df_income['year'] = df_income['year'].astype(str)
df_income = df_income.set_index(['year', 'fips'])
df_main = df_main.join(df_income)

In [7]:
### 5) Pull in deaths by county by year
df_deaths = pd.read_sql('SELECT * FROM ucod_yr', cnx)

df_deaths = df_deaths.rename({'yr': 'year', 'fips_combined_fk': 'fips'}, axis=1)
df_deaths['year'] = df_deaths['year'].astype(str)
df_deaths = df_deaths.set_index(['year', 'fips'])
df_main = df_main.join(df_deaths)

#temp fill death_count with 10 as the min because the ucod_yr db is only counties with more than 10 deaths
df_main['death_count'] = df_main['death_count'].fillna(0)

#create per capita deaths
df_main['deaths_per_100000'] = (df_main['death_count'] / df_main['population']) * 100000

In [8]:
df_main.sort_values(['deaths_per_100000'], ascending=False).head()

state_abbr county_name  population      sales  sales_per_capita  \
year fips                                                                    
2011 54109         WV     Wyoming     23474.0  3182580.0        135.578938   
2010 21197         KY      Powell     12665.0   935180.0         73.839716   
2011 54047         WV    McDowell     21690.0  2242610.0        103.393730   
     21051         KY        Clay     21613.0  3656770.0        169.193078   
2010 54109         WV     Wyoming     23733.0  2860540.0        120.530064   

            median_income  death_count  deaths_per_100000  
year fips                                                  
2011 54109        32851.0         31.0         132.061004  
2010 21197        32041.0         16.0         126.332412  
2011 54047        23751.0         26.0         119.870908  
     21051        22289.0         25.0         115.671124  
2010 54109        35872.0         27.0         113.765643

In [9]:
### 6) Education levels from excel file
usecols = ['FIPS Code', 
           'Percent of adults with less than a high school diploma, 2013-17', 
           'Percent of adults with a high school diploma only, 2013-17', 
           "Percent of adults completing some college or associate's degree, 2013-17", 
           "Percent of adults with a bachelor's degree or higher, 2013-17",
           "2013 Rural-urban Continuum Code", 
           "2013 Urban Influence Code"]
df_education = pd.read_excel('/Users/bovice18/Desktop/GaTech/CSE6242-Team99/data/2013-Education-By-County.xls',
                            skiprows=4, usecols=usecols)

df_education['FIPS Code'] = df_education['FIPS Code'].astype(str)
df_education = df_education.rename(columns={'FIPS Code':'fips'}).set_index(['fips'])

df_main = df_main.join(df_education)

In [10]:
### 7) Poverty levels from excel file
usecols = ['FIPStxt', 
           "PCTPOVALL_2017"]
df_education = pd.read_excel('/Users/bovice18/Desktop/GaTech/CSE6242-Team99/data/2013-PovertyEstimates-By-County.xls',
                            skiprows=3, usecols=usecols)

df_education['FIPStxt'] = df_education['FIPStxt'].astype(str)
df_education = df_education.rename(columns={'FIPStxt':'fips'}).set_index(['fips'])

df_main = df_main.join(df_education)

In [11]:
df_main.sort_values(['deaths_per_100000'], ascending=False).head(10)

state_abbr county_name  population      sales  sales_per_capita  \
year fips                                                                    
2011 54109         WV     Wyoming     23474.0  3182580.0        135.578938   
2010 21197         KY      Powell     12665.0   935180.0         73.839716   
2011 54047         WV    McDowell     21690.0  2242610.0        103.393730   
     21051         KY        Clay     21613.0  3656770.0        169.193078   
2010 54109         WV     Wyoming     23733.0  2860540.0        120.530064   
2011 54005         WV       Boone     24474.0  2962350.0        121.040696   
2010 54047         WV    McDowell     22064.0  2064070.0         93.549220   
2006 22087         LA  St Bernard     14493.0   958970.0         66.167805   
2008 54047         WV    McDowell     22602.0  2149730.0         95.112379   
2012 21131         KY      Leslie     11144.0  1756250.0        157.596016   

            median_income  death_count  deaths_per_100000  \
year fips                                                   
2011 54109        32851.0         31.0         132.061004   
2010 21197        32041.0         16.0         126.332412   
2011 54047        23751.0         26.0         119.870908   
     21051        22289.0         25.0         115.671124   
2010 54109        35872.0         27.0         113.765643   
2011 54005        38357.0         27.0         110.321157   
2010 54047        24133.0         24.0         108.774474   
2006 22087        37375.0         15.0         103.498241   
2008 54047        20486.0         23.0         101.760906   
2012 21131        27517.0         11.0          98.707825   

            2013 Rural-urban Continuum Code  2013 Urban Influence Code  \
year fips                                                                
2011 54109                              6.0                        7.0   
2010 21197                              6.0                        6.0   
2011 54047                              7.0                       10.0   
     21051                              7.0                       10.0   
2010 54109                              6.0                        7.0   
2011 54005                              3.0                        2.0   
2010 54047                              7.0                       10.0   
2006 22087                              1.0                        1.0   
2008 54047                              7.0                       10.0   
2012 21131                              9.0                       10.0   

            Percent of adults with less than a high school diploma, 2013-17  \
year fips                                                                     
2011 54109                                               21.6                 
2010 21197                                               21.1                 
2011 54047                                               34.7                 
     21051                                               37.0                 
2010 54109                                               21.6                 
2011 54005                                               20.9                 
2010 54047                                               34.7                 
2006 22087                                               18.3                 
2008 54047                                               34.7                 
2012 21131                                               31.8                 

            Percent of adults with a high school diploma only, 2013-17  \
year fips                                                                
2011 54109                                               48.4            
2010 21197                                               43.5            
2011 54047                                               44.6            
     21051                                               33.2            
2010 54109                                               48

In [24]:
df_main = df_main.reset_index()
df_main['fips'] = df_main['fips'].apply(str)
df_main['fips'] = df_main['fips'].str.zfill(5)
final = df_main.groupby(['fips']).mean().reset_index()

In [25]:
final.to_csv('~/Downloads/final_tableau.csv')

In [27]:
final['median_income'].mean()

43175.27374614565

In [19]:
final.head()

,fips,population,sales,sales_per_capita,median_income,death_count,deaths_per_100000,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Percent of adults with less than a high school diploma, 2013-17","Percent of adults with a high school diploma only, 2013-17","Percent of adults completing some college or associate's degree, 2013-17","Percent of adults with a bachelor's degree or higher, 2013-17",PCTPOVALL_2017
0,10001,158456.857143,6.364487e+06,39.948796,50762.142857,22.857143,14.231849,3.0,2.0,12.6,33.1,30.8,23.5,13.2
1,10003,535024.000000,2.125049e+07,39.667246,61208.142857,78.857143,14.714964,1.0,1.0,9.0,29.7,25.6,35.7,13.4
2,10005,192760.857143,1.183892e+07,61.085015,48858.142857,28.571429,14.709757,2.0,2.0,13.2,33.4,28.3,25.1,11.9
3,1001,52171.428571,3.038353e+06,57.972863,50703.142857,0.000000,0.000000,2.0,2.0,12.3,33.6,29.1,25.0,13.4
4,1003,179698.857143,9.989512e+06,55.109937,48839.857143,27.714286,15.405228,3.0,2.0,9.8,27.8,31.7,30.7,10.1


In [20]:
len(df_main.reset_index()['fips'].unique())

3235